In [1]:
import os
import cv2 
import numpy as np

In [241]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [238]:
str_a = '0001.png'
img_color = cv2.imread('image0001.png')

In [2]:
img_color.shape

NameError: name 'img_color' is not defined

In [3]:
fname = []

for a in range(1,20):
    fname.append("image"+str(a).zfill(4)+".png")
    
print(fname)

['image0001.png', 'image0002.png', 'image0003.png', 'image0004.png', 'image0005.png', 'image0006.png', 'image0007.png', 'image0008.png', 'image0009.png', 'image0010.png', 'image0011.png', 'image0012.png', 'image0013.png', 'image0014.png', 'image0015.png', 'image0016.png', 'image0017.png', 'image0018.png', 'image0019.png']


In [14]:
def find_balcony(str_name):
    
    img_color = cv2.imread(str_name)
    height, width = img_color.shape[:2]
    img_hsv = cv2.cvtColor(img_color, cv2.COLOR_BGR2HSV)
    
    lower_green = (50-20, 30, 30)
    upper_green = (50+20, 255, 255)

    img_mask = cv2.inRange(img_hsv, lower_green, upper_green)
    img_result = cv2.bitwise_and(img_color, img_color, mask = img_mask)
    img_result_gray = cv2.cvtColor(img_result, cv2.COLOR_HSV2BGR)
    img_result_gray = cv2.cvtColor(img_result_gray, cv2.COLOR_BGR2GRAY)
    
    balcony_num = 0
    isBalcony = 0
    img_Balcony = np.zeros((height, width))
    plane_map_dict = {'balcony' : 0, 'room': 0}

    for row in range(height):
        for col in range(width):
            if img_result_gray[row][col] >70 :
                isBalcony = 1
                img_Balcony[row][col] = isBalcony
    return img_result

In [9]:
def setLabel(img, pts, label):
    (x,y,w,h) = cv2.boundingRect(pts)
    pt1 = (x,y)
    pt2 = (x+w, y+h)
    cv2.rectangle(img, pt1, pt2, (0, 255, 0), 2)
    cv2.putText(img, label, (pt1[0], pt1[1]-3), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255))   

In [10]:
def count_balcony(str_name):
    count = 0
    img = find_balcony(str_name)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ret, thr = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thr, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

    for cont in contours:
        approx = cv2.approxPolyDP(cont, cv2.arcLength(cont, True)* 0.02, True)
        vtc = len(approx)
    
        if vtc > 3:
            setLabel(img, cont, 'balcony')
            count = count + 1
            
    return count
        
# cv2.imshow('img',img)
# # cv2.imshow('binary', thr)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [16]:
count_balcony_list = []
for a in range(len(fname)):
    count_balcony_list.append(count_balcony(fname[a]))

In [17]:
count_balcony_list

[3, 2, 3, 1, 3, 4, 6, 3, 3, 1, 3, 4, 2, 2, 4, 3, 4, 3, 6]

In [19]:
import pandas as pd

df=pd.read_csv('map_info.csv')
df.loc[:, "count_balcony"] = pd.Series(count_balcony_list)

data2.to_csv('final_map_info.csv',header=False, index=False, mode='a')